In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units
from astropy.constants import G
import importlib

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)
import dataset_comp
import snapshot
import curve_fit

In [ ]:
importlib.reload(dataset_comp)
importlib.reload(snapshot)
importlib.reload(curve_fit)

# $M_*$ vs. $v_\text{max}$

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
data = {
    'plain-LCDM': {
        'Snapshot': snapshot.Snapshot('V1_MR_fix', snap_id),
        'M31_identifier': (1, 0),
        'MW_identifier': (2, 0),
        'Color': ['black', 'gray']
    },
    'curv-p082': {
        'Snapshot': snapshot.Snapshot('V1_MR_curvaton_p082_fix', snap_id),
        'M31_identifier': (1, 0),
        'MW_identifier': (1, 1),
        'Color': ['red', 'pink']
    }
}

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data["Snapshot"]
    sm = snap.get_subhalos("Stars/Mass") * units.g.to(units.Msun)
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)
    
    # Split into satellites:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"]
    )

    # Set low mass limit for physical halos:
    mask_vmax = dataset_comp.prune_vmax(snap, low_lim=10)
    
    # Select only luminous:
    mask_lum,_ = dataset_comp.split_luminous(snap)
    mask_m31 = mask_m31[mask_lum]
    mask_mw = mask_mw[mask_lum]
    mask_isol = mask_isol[mask_lum]
    mask_vmax = mask_vmax[mask_lum]
    
    # Add datasets to dictionary:
    data[name]['SM'] = sm[mask_lum]
    data[name]['Vmax'] = vmax[mask_lum]
    data[name]['V1kpc'] = v1kpc[mask_lum]

    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': np.logical_or(mask_m31, mask_mw),
        'Isolated': mask_isol,
        'Physical': mask_vmax
    }

## Plot

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 10,
              'axes.labelsize': 8,
              'xtick.labelsize': 8,
              'ytick.labelsize': 8,
              'legend.fontsize': 8,
              'legend.title_fontsize': 7}

# Marker size
ms = 10

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
# Choose font sizes:
parameters = {'axes.titlesize': 12,
              'axes.labelsize': 10,
              'xtick.labelsize': 9,
              'ytick.labelsize': 9,
              'legend.fontsize': 10}

# Marker size
ms = 15
a = 0.7

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(6, 3))
plt.subplots_adjust(wspace=0.05)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim(10**6, 5*10**9)

axes[0].set_xlim(10, 110)    
axes[1].set_xlim(5, 80)    
axes[0].set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')    
axes[1].set_xlabel('$v_{\mathrm{1 kpc}}[\mathrm{km s^{-1}}]$')

axes[0].set_ylabel('$M_*[\mathrm{M_\odot}]$')

# axes[0].set_title('Satellite galaxies')
# axes[1].set_title('Isolated galaxies')

In [ ]:
# Add scatter plots:
for i, (name, sim_data) in enumerate(data.items()):
    mask = np.logical_and(sim_data["Selections"]["Satellite"],
                          sim_data["Selections"]["Physical"])
    axes[0].scatter(sim_data["Vmax"][mask], sim_data["SM"][mask], alpha=a, marker='+',
                    c=sim_data["Color"][0], s=ms)
    
    mask = np.logical_and(sim_data["Selections"]["Isolated"],
                          sim_data["Selections"]["Physical"])
    axes[0].scatter(sim_data["Vmax"][mask], sim_data["SM"][mask], alpha=a, marker='s',
                    facecolor='none', s=ms, edgecolor=sim_data["Color"][0])
    
fig

In [ ]:
# Add scatter plots:
for i, (name, sim_data) in enumerate(data.items()):
    mask = np.logical_and(sim_data["Selections"]["Satellite"],
                          sim_data["Selections"]["Physical"])
    axes[1].scatter(sim_data["V1kpc"][mask], sim_data["SM"][mask], alpha=a, marker='+',
                    c=sim_data["Color"][0], s=ms)
    
    mask = np.logical_and(sim_data["Selections"]["Isolated"],
                          sim_data["Selections"]["Physical"])
    axes[1].scatter(sim_data["V1kpc"][mask], sim_data["SM"][mask], alpha=a, marker='s',
                    facecolor='none', s=ms, edgecolor=sim_data["Color"][0])
    
fig

In [ ]:
sm_mean = np.mean(
    sim_data["Snapshot"].get_particle_masses(part_type=[4])
) * units.g.to(units.Msun)

# axes[0].axhline(100 * sm_mean, c='gray', linestyle="dotted")
# axes[1].axhline(100 * sm_mean, c='gray', linestyle="dotted")

axes[0].axhline(10**7, c='gray', linestyle="dotted")
axes[1].axhline(10**7, c='gray', linestyle="dotted")

axes[0].scatter([], [], c=data["plain-LCDM"]["Color"][0], alpha=a, marker='+',
                s=ms, label="Satellite")
axes[0].scatter([], [], edgecolor=data["plain-LCDM"]["Color"][0], alpha=a, marker='s',
                facecolor='none', s=ms, label="Isolated")

axes[0].legend()

for name, sim_data in data.items():
    axes[1].scatter([], [], c=sim_data["Color"][0], alpha=a, marker='o',
                    s=ms, label=name)
    
axes[1].legend()

fig

---

## Save the Figure

In [ ]:
filename = "SM_vs_Vmax_vs_V1kpc.png"    
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')